# Imports

In [ ]:
from sklearn.metrics import classification_report
from speech_models import speech_logistic_regression
from process_dataset import speech_features, text_features
from ensemble import SpeechTextEnsemble, StackEnsemble, VoteEnsemble, BlendEnsemble
import numpy as np
import pandas as pd
import pickle

# Methods

In [ ]:
def check_accuracy(model, data_type, save_name=None):
    if data_type == 'text':
        x_train, x_test, y_train, y_test = text_features.get_train_test()
    else:
        x_train, x_test, y_train, y_test = speech_features.get_train_test() 
    
    model.fit(x_train, y_train)

    if save_name != None:
        model.save(save_name)

    results = model.predict(x_test)
    print(classification_report(y_test, results))

def cross_validate(model, data_type, cv, save_name=None):
    if data_type == 'text':
        x, y = text_features.get_data()
    else:
        x, y = speech_features.get_data()

    if save_name != None:
        model.save(save_name)

    scoring = {'accuracy': 'accuracy',
           'f1_macro': 'f1_macro',
           'precision_macro': 'precision_macro',
           'recall_macro' : 'recall_macro'}

    scores = model.cross_validate(x, y, cv=cv, scoring=scoring)
    print('Accuracy: ', np.mean(scores['test_accuracy']))
    print('F1 Macro: ', np.mean(scores['test_f1_macro']))
    print('Precision Macro: ', np.mean(scores['test_precision_macro']))
    print('Recall Macro: ', np.mean(scores['test_recall_macro']))


# Main

In [ ]:
meta_cls = speech_logistic_regression.get_logistic_regression()


In [ ]:
# speech_model = BlendEnsemble(meta_cls=meta_cls, data_type='speech')
speech_model = StackEnsemble(meta_cls=meta_cls, data_type='speech')
# speech_model = VoteEnsemble(type='soft', data_type='speech')
# speech_model = VoteEnsemble(type='hard', data_type='speech')

In [ ]:
# text_model = BlendEnsemble(meta_cls=meta_cls, data_type='text')
text_model = StackEnsemble(meta_cls=meta_cls, data_type='text')
# text_model = VoteEnsemble(type='soft', data_type='text')
# text_model = VoteEnsemble(type='hard', data_type='text')

In [ ]:
combined_model = SpeechTextEnsemble(fit_bases=False)
x_train_s, x_test_s, y_train_s, y_test_s = speech_features.get_train_test()
x_train_t, x_test_t, y_train_t, y_test_t = text_features.get_train_test()


In [ ]:
result = combined_model.predict(x_test_s, x_test_t)


# Result

In [ ]:
print(classification_report(y_test_s, result))